# Configuración de SageMaker Studio

Utiliza este notebook para configurar SageMaker Studio. Solo necesitas ejecutar el código aquí una vez.

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

import os
import sys
from pathlib import Path

CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.append(f"./{CODE_FOLDER}")

DOMAIN_ID=os.environ["DOMAIN_ID"]
USER_PROFILE=os.environ["USER_PROFILE"]

## Paso 1 - Personalizar las Bibliotecas del Kernel

Puedes personalizar SageMaker Studio utilizando las configuraciones del ciclo de vida. Estos son scripts de shell que se activarán mediante eventos del ciclo de vida, como iniciar un nuevo notebook de Studio.

El siguiente script actualiza los paquetes en una Aplicación de Kernel de SageMaker Studio.

In [23]:
%%writefile {CODE_FOLDER}/packages.sh

#!/bin/bash
# This script upgrades the packages on a SageMaker 
# Studio Kernel Application.

set -eux

pip install -q --upgrade pip
pip install -q --upgrade awscli boto3
pip install -q --upgrade scikit-learn==1.3.1
pip install -q --upgrade PyYAML==6.0
pip install -q --upgrade sagemaker
pip install -q --upgrade ipytest

Overwriting code/packages.sh


Ahora podemos crear una nueva configuración de ciclo de vida.

In [24]:
%%bash -s "$DOMAIN_ID" "$USER_PROFILE" "$CODE_FOLDER"

DOMAIN_ID=$(echo "$1")
USER_PROFILE=$(echo "$2")

LCC_CONTENT=`openssl base64 -A -in $3/packages.sh`

aws sagemaker delete-studio-lifecycle-config \
    --studio-lifecycle-config-name ml-seminario

response=$(aws sagemaker create-studio-lifecycle-config \
    --studio-lifecycle-config-name ml-seminario \
    --studio-lifecycle-config-content $LCC_CONTENT \
    --studio-lifecycle-config-app-type KernelGateway) 

arn=$(echo "${response}" | python3 -c "import sys, json; print(json.load(sys.stdin)['StudioLifecycleConfigArn'])")
echo "${arn}"

aws sagemaker update-user-profile --domain-id $DOMAIN_ID \
    --user-profile-name $USER_PROFILE \
    --user-settings '{
        "KernelGatewayAppSettings": {
            "LifecycleConfigArns": ["'$arn'"]
        }
    }'

arn:aws:sagemaker:us-east-1:325223348818:studio-lifecycle-config/ml-school
{
    "UserProfileArn": "arn:aws:sagemaker:us-east-1:325223348818:user-profile/d-givocgtibv1g/default-1682182522641"
}


## Paso 2 - Configurar el Cierre Automático

El siguiente script configura el cierre automático de los kernels inactivos.

In [6]:
%%writefile {CODE_FOLDER}/autoshutdown.sh

#!/bin/bash
# Este script instala la extensión del servidor de comprobación automática de libreta inactiva en SageMaker Studio
# La extensión original tiene una parte de extensión de laboratorio donde los usuarios pueden configurar el tiempo de espera inactivo a través de un widget de Jupyter Lab.
# En esta versión, el script instala solo el lado del servidor de la extensión. El tiempo de espera inactivo
# se puede configurar a través de un script de línea de comandos que también se creará con este create y se colocará en el
# carpeta de inicio del usuario.
#
# La instalación de la extensión del lado del servidor no requiere una conexión a Internet (ya que todas las dependencias se almacenan en el
# paquete de instalación) y se puede realizar a través del modo VPCOnly.

set -eux

# Tiempo de espera en minutos
export TIMEOUT_IN_MINS=60

# Debería estar ejecutándose en el directorio de inicio del usuario, pero solo para comprobar:
cd /home/sagemaker-user

# Al trabajar en un directorio que comienza con ".", no llenaremos la vista de árbol de archivos de Jupyter de los usuarios
mkdir -p .auto-shutdown

# Crea el script de línea de comandos para configurar el tiempo de espera inactivo
cat > .auto-shutdown/set-time-interval.sh << EOF
#!/opt/conda/bin/python
import json
import requests
TIMEOUT=${TIMEOUT_IN_MINS}
session = requests.Session()
# Obteniendo primero el token xsrf de Jupyter Server
response = session.get("http://localhost:8888/jupyter/default/tree")
# llama a la interfaz de la extensión idle_checker para configurar el valor de tiempo de espera
response = session.post("http://localhost:8888/jupyter/default/sagemaker-studio-autoshutdown/idle_checker",
            json={"idle_time": TIMEOUT, "keep_terminals": False},
            params={"_xsrf": response.headers['Set-Cookie'].split(";")[0].split("=")[1]})
if response.status_code == 200:
    print("Succeeded, idle timeout set to {} minutes".format(TIMEOUT))
else:
    print("Error!")
    print(response.status_code)
EOF
chmod +x .auto-shutdown/set-time-interval.sh

# "wget" no forma parte de la imagen base de Jupyter Server, debes instalarlo primero si es necesario para descargar el paquete de instalación.
sudo yum install -y wget
# Puedes descargar el paquete de instalación desde GitHub o, alternativamente, si estás utilizando el modo VPCOnly, puedes alojarlo en S3
wget -O .auto-shutdown/extension.tar.gz https://github.com/aws-samples/sagemaker-studio-auto-shutdown-extension/raw/main/sagemaker_studio_autoshutdown-0.1.5.tar.gz

# O en su lugar, podría servir el paquete de instalación desde un bucket de S3, en cuyo caso no sería necesario "wget":
# aws s3 --endpoint-url [S3 Interface Endpoint] cp s3://[ubicación del paquete de instalación] .auto-shutdown/extension.tar.gz


# Instala la extensión
cd .auto-shutdown
tar xzf extension.tar.gz
cd sagemaker_studio_autoshutdown-0.1.5

# Activa el entorno de Studio solo para instalar la extensión
export AWS_SAGEMAKER_JUPYTERSERVER_IMAGE="${AWS_SAGEMAKER_JUPYTERSERVER_IMAGE:-'jupyter-server'}"
if [ "$AWS_SAGEMAKER_JUPYTERSERVER_IMAGE" = "jupyter-server-3" ] ; then
    eval "$(conda shell.bash hook)"
    conda activate studio
fi;
pip install --no-dependencies --no-build-isolation -e .
jupyter serverextension enable --py sagemaker_studio_autoshutdown
if [ "$AWS_SAGEMAKER_JUPYTERSERVER_IMAGE" = "jupyter-server-3" ] ; then
    conda deactivate
fi;

# Restarts the jupyter server
nohup supervisorctl -c /etc/supervisor/conf.d/supervisord.conf restart jupyterlabserver

# Waiting for 30 seconds to make sure the Jupyter Server is up and running
sleep 30

# Llama al script para configurar el tiempo de espera inactivo y activar la extensión
/home/sagemaker-user/.auto-shutdown/set-time-interval.sh

Overwriting autoshutdown.sh


Ahora podemos crear una nueva configuración de ciclo de vida.

In [ ]:
%%bash -s "$DOMAIN_ID" "$USER_PROFILE" "$CODE_FOLDER"

DOMAIN_ID=$(echo "$1")
USER_PROFILE=$(echo "$2")

LCC_CONTENT=`openssl base64 -A -in $3/autoshutdown.sh`

aws sagemaker delete-studio-lifecycle-config \
    --studio-lifecycle-config-name autoshutdown 2> /dev/null

response=$(aws sagemaker create-studio-lifecycle-config \
    --studio-lifecycle-config-name autoshutdown \
    --studio-lifecycle-config-content $LCC_CONTENT \
    --studio-lifecycle-config-app-type JupyterServer) 

arn=$(echo "${response}" | python3 -c "import sys, json; print(json.load(sys.stdin)['StudioLifecycleConfigArn'])")
echo "${arn}"

aws sagemaker update-user-profile --domain-id $DOMAIN_ID \
    --user-profile-name $USER_PROFILE \
    --user-settings '{
        "JupyterServerAppSettings": {
            "DefaultResourceSpec": {
                "LifecycleConfigArn": "'$arn'",
                "InstanceType": "system"
            },
            "LifecycleConfigArns": ["'$arn'"]
        }
    }'